In [13]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

In [3]:
finance_data = pd.read_csv("credit_card_transactions.csv")
print("Aperçu des données financières :")
print(finance_data.head())

FileNotFoundError: [Errno 2] No such file or directory: 'credit_card_transactions.csv'

In [16]:
print(f"Données financières : {finance_data.shape[0]} lignes et {finance_data.shape[1]} colonnes")

Données financières : 1296675 lignes et 24 colonnes


In [17]:
print(f"Nombre de lignes dans les données financières : {len(finance_data)}")

Nombre de lignes dans les données financières : 1296675


In [18]:
print(finance_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

In [19]:
print(finance_data.describe())

         Unnamed: 0        cc_num           amt           zip           lat  \
count  1.296675e+06  1.296675e+06  1.296675e+06  1.296675e+06  1.296675e+06   
mean   6.483370e+05  4.171920e+17  7.035104e+01  4.880067e+04  3.853762e+01   
std    3.743180e+05  1.308806e+18  1.603160e+02  2.689322e+04  5.075808e+00   
min    0.000000e+00  6.041621e+10  1.000000e+00  1.257000e+03  2.002710e+01   
25%    3.241685e+05  1.800429e+14  9.650000e+00  2.623700e+04  3.462050e+01   
50%    6.483370e+05  3.521417e+15  4.752000e+01  4.817400e+04  3.935430e+01   
75%    9.725055e+05  4.642255e+15  8.314000e+01  7.204200e+04  4.194040e+01   
max    1.296674e+06  4.992346e+18  2.894890e+04  9.978300e+04  6.669330e+01   

               long      city_pop     unix_time     merch_lat    merch_long  \
count  1.296675e+06  1.296675e+06  1.296675e+06  1.296675e+06  1.296675e+06   
mean  -9.022634e+01  8.882444e+04  1.349244e+09  3.853734e+01 -9.022646e+01   
std    1.375908e+01  3.019564e+05  1.284128e+07  5.

In [20]:
print("Valeurs manquantes dans les données financières :")
print(finance_data.isnull().sum())

Valeurs manquantes dans les données financières :
Unnamed: 0                    0
trans_date_trans_time         0
cc_num                        0
merchant                      0
category                      0
amt                           0
first                         0
last                          0
gender                        0
street                        0
city                          0
state                         0
zip                           0
lat                           0
long                          0
city_pop                      0
job                           0
dob                           0
trans_num                     0
unix_time                     0
merch_lat                     0
merch_long                    0
is_fraud                      0
merch_zipcode            195973
dtype: int64


In [ ]:
# colonne de valeurs manquantes
missing_finance = finance_data.isnull().sum()
print("Colonnes avec des valeurs manquantes (Finance) :")
print(missing_finance[missing_finance > 0])

Colonnes avec des valeurs manquantes (Finance) :
merch_zipcode    195973
dtype: int64


#données assurances

In [ ]:
#voir les valeur manquantes dans la colonne 'merch_zipCode'

merch_data = finance_data[finance_data['merch_zipcode'].isnull()]
print("Données avec des valeurs manquantes dans 'merchant' :")
print(merch_data)

Données avec des valeurs manquantes dans 'merchant' :
         Unnamed: 0 trans_date_trans_time            cc_num  \
1                 1   2019-01-01 00:00:44      630423337322   
3                 3   2019-01-01 00:01:16  3534093764340240   
6                 6   2019-01-01 00:04:42    30074693890476   
13               13   2019-01-01 00:07:27  5559857416065248   
14               14   2019-01-01 00:09:03  3514865930894695   
...             ...                   ...               ...   
1296631     1296631   2020-06-21 11:53:11  4430881574719618   
1296636     1296636   2020-06-21 11:55:56  3534330126107879   
1296639     1296639   2020-06-21 11:57:21  4449530933957323   
1296646     1296646   2020-06-21 12:02:13  5359543825610251   
1296670     1296670   2020-06-21 12:12:08    30263540414123   

                                            merchant        category     amt  \
1                    fraud_Heller, Gutmann and Zieme     grocery_pos  107.23   
3                 fraud_Kutch

In [26]:
# Charger les données d'assurance (CSV)
assurance_data = pd.read_csv("freMTPL2freq.csv")
print("\nAperçu des données d'assurance :")
print(assurance_data.head())


Aperçu des données d'assurance :
   IDpol  ClaimNb  Exposure Area  VehPower  VehAge  DrivAge  BonusMalus  \
0    1.0        1      0.10    D         5       0       55          50   
1    3.0        1      0.77    D         5       0       55          50   
2    5.0        1      0.75    B         6       2       52          50   
3   10.0        1      0.09    B         7       0       46          50   
4   11.0        1      0.84    B         7       0       46          50   

  VehBrand   VehGas  Density Region  
0      B12  Regular     1217    R82  
1      B12  Regular     1217    R82  
2      B12   Diesel       54    R22  
3      B12   Diesel       76    R72  
4      B12   Diesel       76    R72  
